In [1]:
!pip install pandas
!pip install pyarrow 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.8/26.8 MB 49.4 MB/s eta 0:00:0000:0100:01


In [4]:
import pandas as pd
import pyarrow.parquet as pq

In [5]:
file_path = 'anonymized_amz_data.parquet'

table = pq.read_table(file_path)

def convert_dbdate(data):
    return data.astype(str)

# 把dbdate改成str，如果需要后续再把这些列改成date类型
converted_columns = {field.name: convert_dbdate(table[field.name]) if 'dbdate' in str(field.type) else table[field.name] for field in table.schema}

df = pd.DataFrame(converted_columns)

In [6]:
df

,PROD_KEY,DIV_DIVISION,DIV_BRAND,DIV_FRANCHISE,DIV_HERO,DIV_PRODUCT_TITLE,BEGIN_DATE,UDS_HASH_KEY,AVERAGE_SALES_PRICE,CHANGE_IN_GLANCE_VIEW_PRIOR_PERIOD,...,AVERAGE_SALES_PRICE_ASIN_CALCULATED,LBB_DOLLAR_LOSS,REPOOS_DOLLAR_LOSS,FTGV_DOLLAR_LOSS,DISPLAY_CLICKS,AFFILIATE_CLICKS,CONVERSION_RATE_CALC,PRODUCT_TYPE_SEARCH,YTD_GLANCE_VIEWS,STANDARD_CVR
0,B08HWR4BKR,CPD,NYX,None,None,NYX PROFESSIONAL MAKEUP Diamonds & Ice Ultimat...,2020-11-29,9207785733392009991,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B01CKOHWB8,PPD,MATRIX,STYLE LINK,None,MATRIX Style Link Mineral Rough Me Up Salt Inf...,2020-11-29,7739900280832403480,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B000EM7962,PPD,MATRIX,VAVOOM STYLING,None,MATRIX Vavoom Freezing Finishing Hairspray | F...,2020-11-29,1414698074695466751,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B0143LU2MC,PPD,MATRIX,TOTAL RESULTS MIRACLE CREATER,None,MATRIX Total Results Miracle Extender Dry Sham...,2020-11-29,2358641804154216551,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B00X5M7G2K,PPD,MATRIX,STYLE LINK,None,MATRIX Style Link Smooth Setter Smoothing Crea...,2020-11-29,3899246270116750937,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014222,B00KAS53FE,CPD,NYX PROFESSIONAL MAKEUP,FRANCHISE UNDEFINED,NON-HERO,NYX PROFESSIONAL MAKEUP Stay Matte But Not Fla...,2020-04-12,6289153155127484953,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.000000
1014223,B007889CCE,CPD,NYX PROFESSIONAL MAKEUP,FRANCHISE UNDEFINED,NON-HERO,NYX PROFESSIONAL MAKEUP Stay Matte But Not Fla...,2020-04-12,6289153155127484953,NaN,47.172413,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,1280.0,1.406250
1014224,B004BLC8MC,LPD,CLARISONIC,FRANCHISE UNDEFINED,NON-HERO,"Clarisonic Mia 1, Sonic Facial Cleansing Brush...",2020-04-12,6289153155127484953,NaN,1.756098,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.000000
1014225,B06XPCY6FS,LPD,CLARISONIC,FRANCHISE UNDEFINED,NON-HERO,Clarisonic Smart Profile Mia 3 & Pedi Plink Us...,2020-04-12,6289153155127484953,NaN,144.000000,...,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.000000


In [13]:
print(df.columns)

Index(['PROD_KEY', 'DIV_DIVISION', 'DIV_BRAND', 'DIV_FRANCHISE', 'DIV_HERO',
       'DIV_PRODUCT_TITLE', 'BEGIN_DATE', 'UDS_HASH_KEY',
       'AVERAGE_SALES_PRICE', 'CHANGE_IN_GLANCE_VIEW_PRIOR_PERIOD',
       ...
       'AVERAGE_SALES_PRICE_ASIN_CALCULATED', 'LBB_DOLLAR_LOSS',
       'REPOOS_DOLLAR_LOSS', 'FTGV_DOLLAR_LOSS', 'DISPLAY_CLICKS',
       'AFFILIATE_CLICKS', 'CONVERSION_RATE_CALC', 'PRODUCT_TYPE_SEARCH',
       'YTD_GLANCE_VIEWS', 'STANDARD_CVR'],
      dtype='object', length=128)


In [14]:
with open('column_names.txt', 'w') as file:
    # Write each column name in a new line
    for column in df.columns:
        file.write(column + '\n')

In [15]:
# Replace 'your_file.xlsx' with the actual path to your XLSX file
file_path = 'sql_samples.xlsx'

# Read the Excel file into a pandas DataFrame
df_excel = pd.read_excel(file_path, header=None)

In [25]:
import re

In [103]:
pattern = r"\bSELECT\s(?:AVG\((\w+)\)|SUM\((\w+)\)|MIN\((\w+)\)|COUNT\((\w+)\)|DISTINCT\((\w+)\)|(\w+))"
col_names = []
for i in range(2, 104):
    query = df_excel.iloc[i, 1]
    matches = re.findall(pattern, query)
    # print(matches)
    result = [next(g for g in match if g != "") for match in matches]
    for col_name in result:
        if col_name != '1' and col_name != 'COUNT' and col_name != 'YEAR' and col_name not in col_names:
            col_names.append(col_name)

In [104]:
pattern = r"\b(?:AVG|MIN|MAX|SUM|COUNT|DISTINCT)\(\s*(\w+)\s*\)|\bselect\s+\w+\s*,\s*(\w+)"
for i in range(2, 104):
    query = df_excel.iloc[i, 1]

    matches = re.findall(pattern, query)
    result = [word for group in matches for word in group if word != '']
    for col_name in result:
        if col_name != '1' and col_name and col_name != 'YEAR' not in col_names:
            col_names.append(col_name)

In [105]:
miss_col_names = []
for col_name in col_names:
    if col_name not in df.columns and col_name not in miss_col_names:
            miss_col_names.append(col_name)

In [106]:
miss_col_names

['AVERAGE_ORDERED_REVENUE_PRICE',
 'SYNDICATED_MONTH',
 'SYNDICATED_YEAR_NAME',
 'CHARGEBACK',
 'NET_PPM_AMZR',
 'SHIPPING_FEE',
 'SELLABLE_OH_INVENTORY']

In [107]:
start, end = 0, len(df_excel.iloc[:, 0])
for i in range(len(df_excel.iloc[:, 0])):
    if df_excel.iloc[i, 0] == "Schema:":
        start = i + 1

In [108]:
col_names_excel = []
for i in range(start, end):
    col_names_excel.append(df_excel.iloc[i, 0])

In [109]:
# col_names_excel

In [110]:
miss_col_names_excel = []
for col_name in df.columns:
    if col_name not in col_names_excel and col_name not in miss_col_names_excel:
            miss_col_names_excel.append(col_name)

In [111]:
miss_col_names_excel

['UDS_HASH_KEY',
 'AVERAGE_SALES_PRICE',
 'NET_PPM',
 'SALES',
 'SALES_BRANDED',
 'SALES_GENERIC',
 'SALES_COMPETETIVE',
 'SALES_AUTO',
 'SALES_NON_BEAUTY',
 'SALES_UNDEFINED',
 'YTD_TOTAL_SALES',
 'CB_CHARGE_AMT',
 'CB_CHARGE_AMT_PRIOR_PERIOD_ACTUAL_VAL',
 'ORIGIN_DATE',
 'REGION',
 'CHANNEL_PARTNER',
 'SOURCE',
 'LOAD_RUN_ID',
 'LOAD_TIMESTAMP',
 'LOAD_DAG_NAME',
 'AVERAGE_SALES_PRICE_ASIN_CALCULATED']

In [112]:
for col_name in col_names:
    if col_name not in col_names_excel:
        print(col_name)

In [114]:
with open('column_names_query.txt', 'w') as file:
    # Write each column name in a new line
    for column in col_names:
        file.write(column + '\n')